### eval testing

## import lib & var setting

In [1]:
# Import Pytorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib for visulization
import matplotlib.pyplot as plt

import sys 

import cv2

import pandas as pd  # 資料分析套件
import numpy as np
import os
from time import time
import re
from timeit import default_timer as timer

import asyncio 

ImportError: cannot import name 'discord_bot_notifier' from 'helperFunction.helperFunctions' (c:\Users\E\Desktop\deepXGB\deepXGB\program\helperFunction\helperFunctions.py)

In [14]:
# ===== var setting   ========

# csv_file = '../data/img_feature.csv'
# csv_file = '../data/img_feature_no_masked.csv'
csv_file = '../data/img_conv2.csv'
# random_seed = 42
# dataset_path = "../HAM10000/base_img_data/"
dataset_path = "../HAM10000/images/"
groundtruth_file = '../HAM10000/GroundTruth.csv'

## prepocessing train & test data

### create name to label dict

### 分出train 跟test的data loader

In [15]:
import random
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from helperFunction.CustomImageDataset import CustomImageDataset 


gray_mean = sum([0.485, 0.456, 0.406]) / 3
gray_std = sum([0.229, 0.224, 0.225]) / 3


# 可以加入轉換
transform64 = torchvision.transforms.Compose([
    transforms.Resize(size=(64,64)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])

transform_std = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_gray_single_channel = transforms.Compose([
    transforms.Resize(256),  # 调整图像大小至256x256
    transforms.CenterCrop(224),  # 中心裁剪至224x224
    transforms.Grayscale(),  # 将图像转换为灰度图
    transforms.ToTensor(),  # 转换为Tensor
    # 归一化，均值和标准差应根据灰度图的实际分布进行调整
    # 这里使用ImageNet预训练模型的均值和标准差的平均值作为示例
    transforms.Normalize(mean=[0.449], std=[0.226]),
])

transform_gray_pretrain_fit = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),  # 将图像转换为单通道灰度图像
    # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 复制灰度通道到三个通道
    # 其他需要的转换，如 Resize, Normalize 等
    transforms.Resize(256),  # 例如，调整图像大小
    transforms.CenterCrop(224),  # 中心裁剪
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 归一化
])

transform_gray_CT_img_fit = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),  # 将图像转换为单通道灰度图像
    # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 复制灰度通道到三个通道
    # 其他需要的转换，如 Resize, Normalize 等
    transforms.Resize(256),  # 例如，调整图像大小
    transforms.CenterCrop(224),  # 中心裁剪
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[gray_mean,gray_mean,gray_mean], std=[gray_std,gray_std,gray_std]),  # 归一化
])




In [16]:
# @transform test unit

# image = Image.open('../data/chestCTData/images/train/adenocarcinoma/000000 (6).png')

# # 打印图像的格式
# print("Format:", image.format)

# # 打印图像的尺寸
# print("Size:", image.size)

# # 打印图像的模式
# print("Mode:", image.mode)

# # 打印图像的额外信息（如果有的话）
# print("Info:", image.info)

# # 将图像转换为NumPy数组
# image_array = np.array(image)

# # 打印图像数组的形状和部分像素值
# print("Image shape:", image_array.shape)
# print("Some pixel values:", image_array[100:110, 100:110])  # 打印左上角5x5区域的像素值

# transformed_img = transform_gray(image)

# print(transformed_img.shape)

In [17]:
sys.path.append(os.path.abspath('..'))

from data.chestCTData.chestCTDataloader import ChestCTDataProcessor
from data.HAM10000.ham10000Dataloader import HAM10000DataProcessor

# dataContainer = ChestCTDataProcessor(transform=transform_gray_single_channel) # for [1,224,224]
# dataContainer = ChestCTDataProcessor(transform=transform_std) # for [3,224,224]
dataContainer = HAM10000DataProcessor(transform=transform_std)

# model_folder_path = '../model/CT chest'
model_folder_path = '../model/HAM10000'

# transform_file_prefix = "/one_channel"
transform_file_prefix = "/three_channel"

train_dataloader , test_dataloader = dataContainer.returnDataloaders()

num_classes = dataContainer.getNumClasses()

# dataContainer.dataPreprocess()

In [18]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, you can cahnge the batch size if you like
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}") # equal to batch siz

Image shape: torch.Size([32, 3, 224, 224]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([32])


In [19]:
# 儲存file list的資訊給XGB訓練時使用
# import json
# with open('test_files_list.json', 'w') as f:
#     json.dump(test_files, f)
# with open('train_files_list.json', 'w') as f:
#     json.dump(train_files, f)

In [20]:
import torch
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Train & test function defination

In [21]:
# Cteate train_step()
def train_step(model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop thourgh data loader data batches
  for batch,(X, y) in enumerate(dataloader):
    # Send data to the target device
    X, y = X.to(device) , y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate the loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5 Optimizer step
    optimizer.step()

    # Calculat accuracy metric
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [22]:
from sklearn.metrics import f1_score
# Create a test step
def test_step(model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        device=device):
  # Put the model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc, all_preds, all_labels = 0, 0, [], []

  with torch.inference_mode():
    # Loop thourgh data loader data batches
    for batch,(X, y) in enumerate(dataloader):
      # Send data to the target device
      X, y = X.to(device) , y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate the loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()


      # Calculate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1) # Maybe same as softmax results?
      test_acc += (test_pred_labels==y).sum().item() / len(test_pred_labels)

      all_preds.extend(test_pred_labels.cpu().numpy())
      all_labels.extend(y.cpu().numpy())

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  # cal F1 score
  f1 = f1_score(all_labels, all_preds, average='weighted')

  return test_loss, test_acc, f1

In [23]:
from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloader + loss functoin
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          best_model_name: str = "best_model.pth",
          device=device,
          enable_early_stop=True):
  # 2. Create empty results dicitonary
  results = {"train_loss": [],
        "train_acc":[],
        "test_loss": [],
        "test_acc":[]}
  
  # init best model loss var
  best_test_loss = float('inf')
  best_model = None
  early_stop_count = 0
  early_stop_limit = 7

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                        dataloader=train_dataloader,
                        loss_fn=loss_fn,
                        optimizer=optimizer,
                        device=device)
    test_loss, test_acc, test_f1 = test_step(model=model,
                      dataloader=test_dataloader,
                      loss_fn=loss_fn,
                      device=device)
    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f} |  F1 score: {test_f1:.4f}   ")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      best_test_acc = test_acc
      best_test_f1 = test_f1
      # best_model = model.state_dict()
      torch.save(model.state_dict(), best_model_name)
      early_stop_count = 0
    else:
      if enable_early_stop:
        early_stop_count += 1
    
    if early_stop_count >= early_stop_limit:
      print("Early stopping triggered.")
      break

  # 6. Return the filled results at the end of the eopchs
      
  # save best model
  # torch.save(best_model, best_model_name)
  print("the best test acc is :{:.4f}".format(best_test_acc))
  print("the best F1 score is :{:.4f}".format(best_test_f1))
  return results

# Train model

## VGG

load preTrained VGG16, num_class = 1000

In [29]:
import torchvision.models as models

model_preTrained_VGG16 = models.vgg16(pretrained=True).to(device)     # 使用內建的 model 

optimizer = torch.optim.SGD(model_preTrained_VGG16.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

# Start the timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_VGG16,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name= model_folder_path + "/best_model_pretrain_VGG16_1000.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 3.1632 | Train acc: 0.3028 | Test loss: 1.3673 | Test acc: 0.4921 |  F1 score: 0.4772   


  2%|▎         | 1/40 [00:16<10:43, 16.51s/it]

Epoch: 1 | Train loss: 1.2012 | Train acc: 0.5963 | Test loss: 1.0539 | Test acc: 0.5698 |  F1 score: 0.5563   


  5%|▌         | 2/40 [00:28<08:53, 14.03s/it]

...

Epoch: 15 | Train loss: 0.1198 | Train acc: 0.9719 | Test loss: 0.7674 | Test acc: 0.6954 |  F1 score: 0.6817   


 42%|████▎     | 17/40 [03:22<04:18, 11.23s/it]

Epoch: 16 | Train loss: 0.1373 | Train acc: 0.9609 | Test loss: 0.8017 | Test acc: 0.6829 |  F1 score: 0.6634   


 42%|████▎     | 17/40 [03:29<04:44, 12.35s/it]


KeyboardInterrupt: 

output == 7 pretrain VGG

In [24]:
import torchvision.models as models

model_preTrained_VGG16 = models.vgg16(pretrained=True)    # 使用內建的 model 


optimizer = torch.optim.SGD(model_preTrained_VGG16.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

#=========
import torch
import torch.nn as nn
import torchvision.models as models




# 获取分类器中最后一个全连接层之前的部分
classifier = list(model_preTrained_VGG16.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 "先前設定的class數"
classifier.append(torch.nn.Linear(4096, num_classes))

# 替换原始模型的分类器
model_preTrained_VGG16.classifier = torch.nn.Sequential(*classifier)

# ============


model_preTrained_VGG16 =  model_preTrained_VGG16.to(device)

from torchinfo import summary

summary(model_preTrained_VGG16, input_size=[1,3,224,224])



# Start the timer


# from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG16_results = train(model=model_preTrained_VGG16,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name=model_folder_path + "\\best_model_pretrain_VGG16.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 1.0898 | Train acc: 0.6409 | Test loss: 0.8946 | Test acc: 0.6916 |  F1 score: 0.6093   


  2%|▎         | 1/40 [10:32<6:51:16, 632.72s/it]

Epoch: 1 | Train loss: 0.8719 | Train acc: 0.6925 | Test loss: 0.8122 | Test acc: 0.7086 |  F1 score: 0.6766   


  5%|▌         | 2/40 [21:41<6:54:04, 653.81s/it]

...

Epoch: 24 | Train loss: 0.3204 | Train acc: 0.8824 | Test loss: 1.1926 | Test acc: 0.6299 |  F1 score: 0.6723   


 65%|██████▌   | 26/40 [4:48:31<2:36:41, 671.57s/it]

Epoch: 25 | Train loss: 0.3254 | Train acc: 0.8836 | Test loss: 0.5848 | Test acc: 0.8016 |  F1 score: 0.7808   


 65%|██████▌   | 26/40 [4:58:32<2:40:44, 688.93s/it]

Epoch: 26 | Train loss: 0.2981 | Train acc: 0.8925 | Test loss: 0.5757 | Test acc: 0.8176 |  F1 score: 0.7945   
Early stopping triggered.
the best test acc is :0.8300
the best F1 score is :0.8196
Total training time: 17912.074 seconds


## Resnet 18

pretrain resnet num_classes = 1000

In [20]:
import torchvision.models as models

model_preTrained_Resnet18 = models.resnet18(pretrained=True).to(device)     # 使用內建的 model 

summary(model_preTrained_Resnet18, input_size=[1,3,224,224])

optimizer = torch.optim.SGD(model_preTrained_Resnet18.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 




# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_pretrain_Resnet18.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  2%|▎         | 1/40 [01:02<40:23, 62.14s/it]

Epoch: 0 | Train loss: 2.4491 | Train acc: 0.5971 | Test loss: 1.2487 | Test acc: 0.6910 |  F1 score: 0.6630   


  5%|▌         | 2/40 [02:04<39:37, 62.56s/it]

Epoch: 1 | Train loss: 0.8927 | Train acc: 0.7424 | Test loss: 0.8683 | Test acc: 0.7302 |  F1 score: 0.7014   


  8%|▊         | 3/40 [03:08<38:43, 62.79s/it]

...

Epoch: 37 | Train loss: 0.0252 | Train acc: 0.9991 | Test loss: 0.7103 | Test acc: 0.7879 |  F1 score: 0.7794   


 98%|█████████▊| 39/40 [41:14<01:06, 66.34s/it]

Epoch: 38 | Train loss: 0.0230 | Train acc: 0.9986 | Test loss: 0.7232 | Test acc: 0.7899 |  F1 score: 0.7801   


100%|██████████| 40/40 [42:20<00:00, 63.52s/it]

Epoch: 39 | Train loss: 0.0211 | Train acc: 0.9984 | Test loss: 0.7264 | Test acc: 0.7878 |  F1 score: 0.7791   
the best test acc is :0.7815
Total training time: 2540.734 seconds


output == 7 pretrain resnet

In [13]:
import torchvision.models as models

model_preTrained_Resnet18 = models.resnet18(pretrained=True)     # 使用內建的 model 


# optimizer = torch.optim.SGD(model_preTrained_Resnet18.parameters(), lr = 0.001) # 選擇你想用的 optimizer
optimizer = torch.optim.SGD(model_preTrained_Resnet18.parameters(), lr = 0.001, momentum=0.9) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

# ========================

# change fully connected layer to 7
num_ftrs = model_preTrained_Resnet18.fc.in_features
model_preTrained_Resnet18.fc = nn.Linear(num_ftrs, num_classes)

# device setting
model_preTrained_Resnet18 = model_preTrained_Resnet18.to(device)

# ===========================

# summary(model_preTrained_Resnet18)


# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name=model_folder_path + "\\best_model_pretrain_Resnet18.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  2%|▎         | 1/40 [00:07<04:36,  7.08s/it]

Epoch: 0 | Train loss: 1.5535 | Train acc: 0.2809 | Test loss: 1.3779 | Test acc: 0.3697 |  F1 score: 0.3204   


  5%|▌         | 2/40 [00:13<04:22,  6.91s/it]

Epoch: 1 | Train loss: 1.0061 | Train acc: 0.5747 | Test loss: 1.1217 | Test acc: 0.4888 |  F1 score: 0.4079   


  8%|▊         | 3/40 [00:20<04:14,  6.87s/it]

...

Epoch: 33 | Train loss: 0.0214 | Train acc: 1.0000 | Test loss: 0.5449 | Test acc: 0.7897 |  F1 score: 0.7896   


 88%|████████▊ | 35/40 [03:50<00:32,  6.51s/it]

Epoch: 34 | Train loss: 0.0325 | Train acc: 0.9984 | Test loss: 0.5343 | Test acc: 0.7897 |  F1 score: 0.7910   


 88%|████████▊ | 35/40 [03:57<00:33,  6.77s/it]

Epoch: 35 | Train loss: 0.0310 | Train acc: 0.9969 | Test loss: 0.5211 | Test acc: 0.7784 |  F1 score: 0.7774   
Early stopping triggered.
the best test acc is :0.7846
the best F1 score is :0.7837
Total training time: 237.045 seconds


### resnet 18 :single channel model

In [38]:
import torch
import torchvision.models as models
from torch import nn, optim

# 加载预训练的ResNet18模型
model_resnet18 = models.resnet18(pretrained=True)

# 修改第一层卷积以接受单通道输入
# ResNet18的第一层卷积的输入通道数默认为3，我们需要将其更改为1
model_resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# 如果需要，冻结模型的前几层参数
for param in model_resnet18.parameters():
    param.requires_grad = False
# 例如，解冻最后的卷积层
for param in model_resnet18.layer4.parameters():
    param.requires_grad = True

# 修改分类器以适应新的数据集
# ResNet18使用全连接层作为其分类器，我们需要根据目标数据集的类别数来调整它
num_ftrs = model_resnet18.fc.in_features
model_resnet18.fc = nn.Linear(num_ftrs, 10)  # 假设有10个类

# 将模型移动到适当的设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet18 = model_resnet18.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# 确保只更新需要梯度的参数
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_resnet18.parameters()), lr=0.001, momentum=0.9)

# 接下来，您需要编写适合您数据集的训练和验证代码...



# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_resnet18,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name=model_folder_path + "\\best_model_pretrain_Resnet18_oneC.pth",
              epochs=40,
              enable_early_stop=False)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/40 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:05<03:50,  5.90s/it]

Epoch: 0 | Train loss: 1.4953 | Train acc: 0.4341 | Test loss: 1.1949 | Test acc: 0.4763 |  F1 score: 0.4900   


  5%|▌         | 2/40 [00:11<03:37,  5.73s/it]

Epoch: 1 | Train loss: 0.7456 | Train acc: 0.6997 | Test loss: 0.9443 | Test acc: 0.5856 |  F1 score: 0.5661   


...

Epoch: 37 | Train loss: 0.0126 | Train acc: 0.9969 | Test loss: 0.6418 | Test acc: 0.7590 |  F1 score: 0.7506   


 98%|█████████▊| 39/40 [03:39<00:05,  5.61s/it]

Epoch: 38 | Train loss: 0.0116 | Train acc: 0.9984 | Test loss: 0.6340 | Test acc: 0.7622 |  F1 score: 0.7560   


100%|██████████| 40/40 [03:45<00:00,  5.63s/it]

Epoch: 39 | Train loss: 0.0098 | Train acc: 0.9984 | Test loss: 0.7155 | Test acc: 0.7368 |  F1 score: 0.7300   
the best test acc is :0.7471
the best F1 score is :0.7401
Total training time: 225.232 seconds


### resnet 50

In [17]:
import torchvision.models as models

model_preTrained_Resnet50 = models.resnet50(pretrained=True)     # 使用內建的 model 


# optimizer = torch.optim.SGD(model_preTrained_Resnet50.parameters(), lr = 0.001) # 選擇你想用的 optimizer
optimizer = torch.optim.SGD(model_preTrained_Resnet50.parameters(), lr = 0.001, momentum=0.9) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet50.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 


# change fully connected layer to 7
num_ftrs = model_preTrained_Resnet50.fc.in_features
model_preTrained_Resnet50.fc = nn.Linear(num_ftrs, num_classes)

# device setting
model_preTrained_Resnet50 = model_preTrained_Resnet50.to(device)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet50,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name=model_folder_path + "\\best_model_pretrain_Resnet50_7.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  2%|▎         | 1/40 [00:09<06:20,  9.77s/it]

Epoch: 0 | Train loss: 1.3120 | Train acc: 0.4153 | Test loss: 1.2851 | Test acc: 0.4382 |  F1 score: 0.3769   


  5%|▌         | 2/40 [00:19<06:00,  9.48s/it]

Epoch: 1 | Train loss: 1.0068 | Train acc: 0.6953 | Test loss: 1.0484 | Test acc: 0.5712 |  F1 score: 0.5235   


  8%|▊         | 3/40 [00:28<05:46,  9.35s/it]

...

Epoch: 28 | Train loss: 0.0365 | Train acc: 0.9984 | Test loss: 0.5818 | Test acc: 0.7741 |  F1 score: 0.7719   


 75%|███████▌  | 30/40 [05:12<02:06, 12.69s/it]

Epoch: 29 | Train loss: 0.0194 | Train acc: 0.9984 | Test loss: 0.5773 | Test acc: 0.7797 |  F1 score: 0.7796   


 75%|███████▌  | 30/40 [05:24<01:48, 10.81s/it]

Epoch: 30 | Train loss: 0.0238 | Train acc: 0.9969 | Test loss: 0.5437 | Test acc: 0.8096 |  F1 score: 0.8088   
Early stopping triggered.
the best test acc is :0.7834
the best F1 score is :0.7840
Total training time: 324.209 seconds


### resnet 101

In [30]:
import torchvision.models as models
from torchinfo import summary

model_Resnet101 = models.densenet201(pretrained=True)     # 使用內建的 model 

summary(model_Resnet101, input_size=[1,3,224,224])

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to C:\Users\E/.cache\torch\hub\checkpoints\densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:01<00:00, 71.9MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
DenseNet                                 [1, 1000]                 --
├─Sequential: 1-1                        [1, 1920, 7, 7]           --
│    └─Conv2d: 2-1                       [1, 64, 112, 112]         9,408
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         128
│    └─ReLU: 2-3                         [1, 64, 112, 112]         --
│    └─MaxPool2d: 2-4                    [1, 64, 56, 56]           --
│    └─_DenseBlock: 2-5                  [1, 256, 56, 56]          --
│    │    └─_DenseLayer: 3-1             [1, 32, 56, 56]           45,440
│    │    └─_DenseLayer: 3-2             [1, 32, 56, 56]           49,600
│    │    └─_DenseLayer: 3-3             [1, 32, 56, 56]           53,760
│    │    └─_DenseLayer: 3-4             [1, 32, 56, 56]           57,920
│    │    └─_DenseLayer: 3-5             [1, 32, 56, 56]           62,080
│    │    └─_DenseLayer: 3-6             [1, 32, 56, 56]     

# model output

In [22]:
import torchvision.models as models

new_load_model = models.resnet18(pretrained=True)     # 使用內建的 model 

optimizer = torch.optim.SGD(new_load_model.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 




# VGG16 setting
# =============
# # 获取分类器中最后一个全连接层之前的部分
# classifier = list(new_load_VGG16.classifier.children())[:-1]

# # 移除原始模型的最后一个全连接层
# # 并添加一个新的全连接层，输出特征数为 7
# classifier.append(torch.nn.Linear(4096, 7))

# # 替换原始模型的分类器
# new_load_VGG16.classifier = torch.nn.Sequential(*classifier)

# ============

# resnet setting
#  ============
# change fully connected layer to 7
num_ftrs = new_load_model.fc.in_features
new_load_model.fc = nn.Linear(num_ftrs, 7)
#  ============


new_load_model =  new_load_model.to(device)

new_load_model.load_state_dict(torch.load("./best_model_pretrain_Resnet18_7.pth", map_location=device))

from torchinfo import summary

# summary(model_preTrained_VGG16_7, input_size=[1,3,224,224])





# Start the timer
from timeit import default_timer as timer
start_time = timer()

new_load_model.eval()

loss, acc, f1 = test_step(new_load_model,
                        loss_fn=loss_fn,
                        dataloader=test_data_loader_224,
                        device=device)

print("acc:{:.4f}".format(acc))
print("f1:{:.4f}".format(f1))



c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


acc:0.8312
f1:0.8258


In [21]:
print(loss)
print("acc:{:.4f}".format(acc))
print("f1:{:.4f}".format(f1))


0.6782166653058745
acc:0.7466
f1:0.7649


In [26]:
# image_path = "./HAM10000/images/ISIC_0024306.jpg"

# def img2tensor(img_path):
#     img = cv2.imread(img_path)
#     img = cv2.resize(img, (224, 224))

#     data = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
#     return data

# def flattenTensor(tensor):
#     tensor = torch.squeeze(tensor)
#     flatted_tensor = tensor.view(-1)
#     return flatted_tensor

# # output = conv_layer(input_data)
# # print(input_data.shape)
# # print(output.shape)

# # output = conv_block(input_data)
# # output = conv_block_2(output)
# input_data = img2tensor(image_path)
# input_data = input_data.to(device)

# model_VGG16.eval()
# with torch.no_grad():
#     output = model_VGG16(input_data)
# prob = torch.softmax(output, dim=1)
# print(input_data.shape)
# print(output.shape)
# print(prob)
# print(flattenTensor(output).shape)

In [27]:
# import os
# import csv
# import time

# input_folder = "HAM10000/images/"
# csvfile_name = "./img_conv3.csv"


# res = []

# # open every img
# start = time.time()
# for filename in os.listdir(input_folder):

#     image_path  = os.path.join(input_folder, filename)
#     # print(image_path)
#     img = img2tensor(image_path)
#     tensor = conv3(img)
#     tensor = flattenTensor(tensor)
#     vec = tensor.tolist()

#     vec = [filename] + vec
#     res.append(vec)
# end = time.time()

# print("time cost : {}".format(end - start))

# # print(res)
# with open(csvfile_name, 'w', newline='', encoding='utf-8') as csvfile:
#     # try:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerows(res)
#     # except Exception as e:
#     #     logging.error("csv error : %s" , e)
#     #     print('csv encoding error')

## Tiny VGG

input wiil be [3, 64, 64]

In [28]:
# # Set random seeds

# torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# # Set number of epochs
# NUM_EPOCHS = 30

# # Recreate an instance of TinyVGG
# model_0 = TinyVGG(input_shape=3, # numbers of color channels
#           hidden_units=10,
#           output_shape=7).to(device)

# # Setup loss function and optimizer
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params=model_0.parameters(),
#                 lr =0.001)

# # Start the timer
# from timeit import default_timer as timer
# start_time = timer()

# # Train model_0
# model_0_results = train(model=model_0,
#               train_dataloader=train_data_loader,
#               test_dataloader=test_data_loader,
#               optimizer=optimizer,
#               loss_fn=loss_fn,
#               epochs=NUM_EPOCHS)

# # End the timer and print out how long it took
# end_time = timer()
# print(f"Total training time: {end_time-start_time:.3f} seconds")


 83%|████████▎ | 25/30 [25:30<05:04, 60.83s/it]
Epoch: 24 | Train loss: 0.5824 | Train acc: 0.7830 | Test loss: 0.8261 | Test acc: 0.7118  
 87%|████████▋ | 26/30 [26:31<04:03, 60.86s/it]
Epoch: 25 | Train loss: 0.5774 | Train acc: 0.7839 | Test loss: 0.8404 | Test acc: 0.7162  
 90%|█████████ | 27/30 [27:32<03:02, 60.86s/it]
Epoch: 26 | Train loss: 0.5776 | Train acc: 0.7862 | Test loss: 0.8416 | Test acc: 0.7101  
 93%|█████████▎| 28/30 [28:33<02:01, 60.82s/it]
Epoch: 27 | Train loss: 0.5675 | Train acc: 0.7873 | Test loss: 0.8176 | Test acc: 0.7065  
 97%|█████████▋| 29/30 [29:34<01:00, 60.79s/it]
Epoch: 28 | Train loss: 0.5592 | Train acc: 0.7901 | Test loss: 0.8350 | Test acc: 0.7147  
100%|██████████| 30/30 [30:35<00:00, 61.17s/it]
Epoch: 29 | Train loss: 0.5536 | Train acc: 0.7954 | Test loss: 0.8560 | Test acc: 0.7035  
Total training time: 1835.003 seconds
